In [4]:
import pandas as pd 
import matplotlib.pyplot as plt
import math
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import tensorflow as tf
import datetime




In [5]:
# Bring the dataset into the code
df = pd.read_csv("bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv")
df.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Remove  "NaN" values from the dataset
df = df.dropna()
df.head()


In [ ]:
# Convert the 'Timestamp' column to datetime type
df['Date'] = pd.to_datetime(df['Timestamp'], unit='s')

# Create new columns 'Date' and 'Time' using the 'Date' column
df['Time'] = df['Date'].dt.time
df['Date']=df['Date'].dt.floor("D")

df

In [ ]:
# Visualize the data
plt.figure(figsize=(30, 10))
plt.title("Close Price History", fontsize=40)
plt.plot(df["Date"], df["Close"], label='Close Price')
plt.xlabel("Date", fontsize=28)
plt.ylabel("Close Price USD", fontsize=28)
plt.legend(fontsize=30) 
plt.show()

In [ ]:
# Create a new dataset containing only the "Close" column
data=df.filter(["Close"])
# Convert the dataset to a numpy array
dataset=data.values
# Calculate the training data length as 80% of the dataset length
training_data_len=math.ceil(len(dataset) * .8)
training_data_len

In [ ]:
# Create MinMaxScaler with feature range between 0 and 1
scaler=MinMaxScaler(feature_range=(0,1))

# Fit and transform the dataset using the scaler
scaled_data = scaler.fit_transform(dataset)
scaled_data


In [ ]:
# Create the training dataset
train_data = scaled_data[0:training_data_len, :]

# Split the data into x_train and y_train sets
x_train=[]
y_train=[]
for i in range(60, len(train_data)):
    x_train.append(np.array(train_data[i-60:i, 0]))
    y_train.append(train_data[i, 0])
    if i <= 60:
        print(x_train)
        print(y_train)
        print()

In [ ]:
# Convert x_train and y_train sets to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)



In [ ]:
# Reshape x_train
x_train = np.reshape(x_train,(x_train.shape[0], x_train.shape[1], 1 ))
x_train.shape

In [ ]:
#Create LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(50,return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
#Train the model
model.fit(x_train, y_train, batch_size=1, epochs=1)

In [ ]:
# Extract x_test from test_data
test_data = scaled_data[training_data_len - 60:, :]
x_test=[]
# Extract y_test from dataset
y_test=dataset[training_data_len:, :]
for i in range (60, len(test_data)):
    x_test.append(test_data[i-60:i,0])

In [ ]:
# Convert x_test to a numpy array
x_test=np.array(x_test)

In [ ]:
x_test=np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))

In [ ]:
# Make predictions on the test data and inverse transform to original scale
predictions=model.predict(x_test)
predictions=scaler.inverse_transform(predictions)

In [ ]:
# Calculate the Root Mean Squared Error (RMSE)
rmse = np.sqrt(np.mean((predictions - y_test)**2))
rmse

In [ ]:
# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Create a new column containing the 'Year' information
df['Year'] = df['Date'].dt.year

# Group the data by the 'Year' column and visualize the results
plt.figure(figsize=(20,10))
plt.title("Model")
plt.xlabel("Date", fontsize=20)
plt.ylabel("Close Price USD", fontsize=20)

# Plot the training data
plt.plot(train['Date'], train['Close'], label='Train')

# Plot the validation data
plt.plot(valid['Date'], valid['Close'], label='Validation')

# Plot the predictions
plt.plot(valid['Date'], valid['Predictions'], label='Predictions')

plt.legend()
plt.show()

In [ ]:
#Get the quote
btc_quote=pd.read_csv("bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv")
btc_quote=btc_quote.dropna()
#Create new dataframe
new_df=btc_quote.filter(["Close"])
#Get the last 60 day closing price values and convert the dataframe to an array
last_60_days=new_df[-60:].values
#Scale the data to be values between 0 and 1 
last_60_days_scaled=scaler.transform(last_60_days)
#Create an empty list
X_test=[]
#Append the past 60 days 
X_test.append(last_60_days_scaled)
#Convert the X_test data set to a numpy array
X_test=np.array(X_test)
#Reshape the data 
X_test = np.reshape(X_test,(X_test.shape[0], X_test.shape[1],1))
#Get the predicted scaled price
pred_price = model.predict(X_test)
#Undo the scaling
pred_price = scaler.inverse_transform(pred_price)
print(pred_price)

In [ ]:
def predict_future_price(model, scaler, last_60_days, target_date):
    # Scale the last 60 days
    last_60_days_scaled = scaler.transform(last_60_days)

    # Create input data for prediction
    X_test = []
    X_test.append(last_60_days_scaled)
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    # Make predictions using the model
    pred_price = model.predict(X_test)

    # Inverse the scaling
    pred_price = scaler.inverse_transform(pred_price)

    # Create the prediction date
    target_date = pd.to_datetime(target_date)
    target_date_str = target_date.strftime("%Y-%m-%d")

    # Print the prediction
    print(f"Predicted closing price for {target_date_str}: {pred_price[0][0]}")

# Example of usage
target_date = input("Write the target date ")  # Hedef tarihi belirtin
last_60_days_for_prediction = df[df['Date'] < target_date].tail(60)  # Hedef tarih öncesindeki son 60 günü alın

# Fonksiyonu kullanarak tahmin yapın
predict_future_price(model, scaler, last_60_days_for_prediction['Close'].values.reshape(-1, 1), target_date)